# Chr17-22, GTEx Adipose (subcutaneous) fusion weights, n=80k

## Part1: simulation and mr.ash

* set up

In [1]:
wkdir = "/project2/mstephens/causalTWAS/simulations/simulation_ashtest_20200730/"

In [2]:
codedir = "/project2/mstephens/causalTWAS/causal-TWAS/code/"

In [3]:
geno = "/project2/mstephens/causalTWAS/ukbiobank/ukb_chr17to22_s80000"

In [4]:
weight = "/project2/mstephens/causalTWAS/fusion_weights/Adipose_Subcutaneous"

In [5]:
maintag = '20200730-1'

In [6]:
subtags =  [str(i) for i in [2,3]]

* prepare param 

In [7]:
cd {wkdir}

/project2/mstephens/causalTWAS/simulations/simulation_ashtest_20200730


In [8]:
template = "{wkdir}/param-{maintag}.R".format(wkdir = wkdir, maintag = maintag)

In [9]:
a = open(template, 'r').readlines()

In [10]:
for subtag in subtags:
    with open("param-{maintag}-{subtag}.R".format(maintag = maintag, subtag = subtag), 'w') as newpar:
        a[5] = 'SED <- {sed}\n'.format(sed = str(950 + int(subtag)))
        newpar.writelines(a)

* commands

In [11]:
g2dir = "cd {wkdir}"

In [12]:
simu = "Rscript {codedir}/run_simulate_data.R {geno} {weight} param-{tag}.R simu_{tag}"

In [13]:
simu_expr = "{wkdir}/simu_{tag}-cis-expr.Rd"

In [14]:
simu_pheno = "{wkdir}/simu_{tag}-pheno.Rd"

In [15]:
mrash2 = "Rscript {codedir}/run_test_mr.ash2s.R {geno} {simu_expr} {simu_pheno} {tag}"

In [16]:
exprgwas = "Rscript {codedir}/run_gwas_expr.R {simu_expr} {simu_pheno} {tag}"

In [17]:
snpgwas = "Rscript {codedir}/run_gwas_snp.R {geno} {simu_pheno} {tag}"

In [18]:
ld = "Rscript {codedir}/run_ld.R {geno} {simu_pheno} {tag}"

* write master file

In [19]:
tags = [maintag + '-' + subtag for subtag in subtags]

In [20]:
masterfile = 'master_run1-x.sh'

In [21]:
cd {wkdir}

/project2/mstephens/causalTWAS/simulations/simulation_ashtest_20200730


In [22]:
with open(masterfile, 'w') as outf:
    for tag in tags:
        outf.write(';'.join([
              'source activate twas', 
              g2dir,
              #simu,
              mrash2,
              #exprgwas,
              #snpgwas
              #ld
        ]).format(
              simu_expr = simu_expr,
              simu_pheno = simu_pheno,
              wkdir = wkdir,
              codedir = codedir,
              geno = geno,
              weight = weight,
              tag = tag
        ).format(
              wkdir = wkdir,
              tag = tag
        ) + '\n')

In [23]:
!python /home/simingz/cancer_somatic/cancer_somatic/annotation/pMake_batchjobs_RCC.py {masterfile} --mem 80 --ntasks 6

planning to run 2 jobs ...
Type: sbatch submit_parallel_master_run1-x.sh.sh
